In [7]:
import pandas as pd
import time
import datetime
import os
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats, playerawards




In [11]:
df = pd.read_csv("playerdata.csv")
df["height"]


0      203.20
1      210.82
2      205.74
3      195.58
4      213.36
        ...  
567    185.42
568    213.36
569    203.20
570    203.20
571    208.28
Name: height, Length: 572, dtype: float64

In [13]:
east_teams = [
    'Celtics', 'Nets', 'Knicks', '76ers', 'Raptors',
    'Bulls', 'Cavaliers', 'Pistons', 'Pacers', 'Bucks',
    'Heat', 'Hawks', 'Hornets', 'Magic', 'Wizards'
]
df['conference'] = df['team'].apply(lambda team: 'East' if team in east_teams else 'West')

In [15]:
def ask_question(question: str):
    while True:
        ans = input(f"{question} (y/n): ").strip().lower()
        if ans in ['y', 'yes']:
            return True
        elif ans in ['n', 'no']:
            return False
        else:
            print("Please answer with 'y' or 'n'.")

def cm_to_inches(cm):
    inches = cm / 2.54
    feet = int(inches // 12)
    inches_remainder = round(inches % 12)
    return f"{feet}'{inches_remainder}\""

def clean_position(pos):
    pos = str(pos).lower()
    if 'guard' in pos and 'forward' in pos:
        return 'G-F'
    if ('forward' in pos and 'center' in pos) or ('center' in pos and 'forward' in pos):
        return 'F-C'
    if 'forward' in pos:
        return 'F'
    if 'center' in pos:
        return 'C'
    if 'guard' in pos:
        return 'G'
    return 'Unknown'
print("🔮 Think of an active NBA player from the dataset...")
print("Let's narrow down who your player is!")

df = pd.read_csv("playerdata.csv")
df['awards'] = df['awards'].fillna('')
df['awards_count'] = df['awards'].apply(lambda x: len(x.split(',')) if x else 0)
df['position'] = df['position'].apply(clean_position)
is_east_conference = ask_question("Is your player in the Eastern Conference?")
chosen_conference = "East" if is_east_conference else "West"
players = df[df['conference'] == chosen_conference].copy()
def find_player(players_df):
    current_players = players_df.copy()
    questions_asked = 0
    while len(current_players) > 1:
        questions_asked += 1
        print(f"[Debug] {len(current_players)} players remaining.")
        if len(current_players) <= 5:
            for _, player_row in current_players.iterrows():
                player_name = player_row['full_name']
                is_player = ask_question(f"Is your player {player_name}?")
                if is_player:
                    return player_name
            return "Player not found. There might be an error in the dataset."
        best_column = None
        best_threshold = None
        best_balance = float('inf')  
        teams = current_players['team'].unique()
        if len(teams) > 1:
            for team in teams:
                team_count = (current_players['team'] == team).sum()
                non_team_count = len(current_players) - team_count
                balance = abs(team_count - non_team_count)
                
                if balance < best_balance:
                    best_balance = balance
                    best_column = 'team'
                    best_threshold = team
        position_groups = {
            'pure_guard': current_players['position'] == 'G',
            'pure_forward': current_players['position'] == 'F',
            'pure_center': current_players['position'] == 'C',
            'guard_forward': current_players['position'] == 'G-F',
            'forward_center': current_players['position'] == 'F-C'
        }
        
        for pos_type, pos_filter in position_groups.items():
            pos_count = pos_filter.sum()
            non_pos_count = len(current_players) - pos_count
            balance = abs(pos_count - non_pos_count)
            
            if balance < best_balance and pos_count > 0:
                best_balance = balance
                best_column = 'position_specific'
                best_threshold = pos_type
        has_awards = current_players['awards_count'] > 0
        award_count = has_awards.sum()
        no_award_count = len(current_players) - award_count
        award_balance = abs(award_count - no_award_count)
        
        if award_balance < best_balance:
            best_balance = award_balance
            best_column = 'awards'
            best_threshold = None
        numeric_cols = ['age', 'height', 'weight', 'average_points', 'average_assists', 
                       'average_rebounds', 'average_steals', 'average_blocks']
        
        for col in numeric_cols:
            values = sorted(current_players[col].unique())
            if len(values) > 1:
                for i in range(len(values) - 1):
                    threshold = (values[i] + values[i + 1]) / 2
                    
                    left_count = (current_players[col] <= threshold).sum()
                    right_count = len(current_players) - left_count
                    balance = abs(left_count - right_count)
                    
                    if balance < best_balance:
                        best_balance = balance
                        best_column = col
                        best_threshold = threshold
        if best_column == 'team':
            answer = ask_question(f"Is your player on the {best_threshold}?")
            if answer:
                current_players = current_players[current_players['team'] == best_threshold]
            else:
                current_players = current_players[current_players['team'] != best_threshold]
        
        elif best_column == 'position_specific':
            if best_threshold == 'pure_guard':
                answer = ask_question(f"Is your player strictly a Guard (G), not a Guard-Forward hybrid?")
                if answer:
                    current_players = current_players[current_players['position'] == 'G']
                else:
                    current_players = current_players[current_players['position'] != 'G']
            elif best_threshold == 'pure_forward':
                answer = ask_question(f"Is your player strictly a Forward (F), not a hybrid position?")
                if answer:
                    current_players = current_players[current_players['position'] == 'F']
                else:
                    current_players = current_players[current_players['position'] != 'F']
            elif best_threshold == 'pure_center':
                answer = ask_question(f"Is your player strictly a Center (C), not a Forward-Center hybrid?")
                if answer:
                    current_players = current_players[current_players['position'] == 'C']
                else:
                    current_players = current_players[current_players['position'] != 'C']
            elif best_threshold == 'guard_forward':
                answer = ask_question(f"Is your player a Guard-Forward (G-F) hybrid?")
                if answer:
                    current_players = current_players[current_players['position'] == 'G-F']
                else:
                    current_players = current_players[current_players['position'] != 'G-F']
            elif best_threshold == 'forward_center':
                answer = ask_question(f"Is your player a Forward-Center (F-C) hybrid?")
                if answer:
                    current_players = current_players[current_players['position'] == 'F-C']
                else:
                    current_players = current_players[current_players['position'] != 'F-C']
        
        elif best_column == 'awards':
            answer = ask_question(f"Has your player received any awards?")
            if answer:
                current_players = current_players[current_players['awards_count'] > 0]
            else:
                current_players = current_players[current_players['awards_count'] == 0]
        
        elif best_column == 'age':
            answer = ask_question(f"Is your player older than {int(best_threshold)} years?")
            if answer:
                current_players = current_players[current_players[best_column] > best_threshold]
            else:
                current_players = current_players[current_players[best_column] <= best_threshold]
        
        elif best_column == 'height':
            answer = ask_question(f"Is your player taller than {cm_to_inches(best_threshold)}?")
            if answer:
                current_players = current_players[current_players[best_column] > best_threshold]
            else:
                current_players = current_players[current_players[best_column] <= best_threshold]
        
        elif best_column == 'weight':
            answer = ask_question(f"Is your player heavier than {int(best_threshold)} lbs?")
            if answer:
                current_players = current_players[current_players[best_column] > best_threshold]
            else:
                current_players = current_players[current_players[best_column] <= best_threshold]
        
        elif best_column in ['average_points', 'average_assists', 'average_rebounds', 'average_steals', 'average_blocks']:
            stat_name = best_column.replace('average_', '')
            answer = ask_question(f"Does your player average more than {best_threshold:.1f} {stat_name}?")
            if answer:
                current_players = current_players[current_players[best_column] > best_threshold]
            else:
                current_players = current_players[current_players[best_column] <= best_threshold]
        else:
            half_index = len(current_players) // 2
            first_half = current_players.iloc[:half_index]
            player_names = ', '.join(first_half['full_name'].values)
            
            answer = ask_question(f"Is your player one of these: {player_names}?")
            if answer:
                current_players = first_half
            else:
                current_players = current_players.iloc[half_index:]
    if len(current_players) == 1:
        player_name = current_players['full_name'].iloc[0]
        confirmation = ask_question(f"Just to confirm: Is your player {player_name}?")
        if confirmation:
            return player_name
        else:
            print("Hmm, something seems off. Let me try a different approach.")
            return find_player_with_direct_questions(players_df)
    else:
        return "Could not determine the player. Something went wrong."

def find_player_with_direct_questions(players_df):
    """Fallback function that just asks directly about each player."""
    print("\n⚠️ Using backup approach to find your player...")
    
    for team in sorted(players_df['team'].unique()):
        team_players = players_df[players_df['team'] == team]
        if ask_question(f"Is your player on the {team}?"):
            print(f"Great! Your player is on the {team}.")
            for _, player in team_players.iterrows():
                player_name = player['full_name']
                if ask_question(f"Is your player {player_name}?"):
                    return player_name
            return "Player not found in the dataset for the selected team."
    
    return "Player not found in the dataset."
guess = find_player(players)
print(f"\n🏀 Mmmm, My guess is... **{guess}**!")
print("Thanks for playing!")

🔮 Think of an active NBA player from the dataset...
Let's narrow down who your player is!
[Debug] 304 players remaining.
[Debug] 149 players remaining.
[Debug] 75 players remaining.
[Debug] 38 players remaining.
[Debug] 19 players remaining.
[Debug] 10 players remaining.
[Debug] 5 players remaining.

🏀 My guess is... **Player not found. There might be an error in the dataset.**!
Thanks for playing!
